# Agent 구축 샘플
Langchain 주요 사용 기법인 에이전트 구축을 기술한다.

## 라이브러리 설치
Langchain 사용을 위한 필수 라이브러리를 설치한다.

In [7]:
!pip install langchain langchain-openai langchain-community langgraph langchain-anthropic tavily-python


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## LangSmith 연동
LangSmith를 통해 흐름을 분석하길 원한다면 아래 변수를 설정하여 연동한다.

In [4]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "Insert LangSmith API Key"

## Tavily
검색 엔진 도구로 [Tavily](https://app.tavily.com/home)를 사용하기 위해 API키를 설정한다.

In [5]:
os.environ["TAVILY_API_KEY"]="Insert Tavily API Key"

### Tool을 정의합니다.
에이전트가 사용할 툴을 생성해야한다. LangChain에 내장되어있는 tavily_search 툴을 사용한다.

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Seoul")
print(search_results)
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718699929, 'localtime': '2024-06-18 1:38'}, 'current': {'last_updated_epoch': 1718699400, 'last_updated': '2024-06-18 01:30', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 70, 'cloud': 25, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 11.9, 'windchill_f': 53.4, 'heatindex_c': 12.2, 'heatindex_f': 53.9, 'dewpoint_c': 7.9, 'dewpoint_f': 46.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 7.3, 'gust_kph': 11.8}}"}, {'url': 'https://www.wunderground.com/hourly/us/ca

## LLM 정의
LangChain에서 다양한 언어 모델을 지원하지만 여기서는 OpenAI 모델을 사용한다.

In [8]:
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "Insert OpenAI API Key"
model = ChatOpenAI(model="gpt-4o")

언어 모델을 호출할 때 메시지 목록을 전달한다.

In [9]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

`.bind_tools`를 사용하여 언어 모델이 툴에 대한 지식을 갖도록 한다.

In [10]:
model_with_tools = model.bind_tools(tools)

모델을 호출하기 위해 일반 메시지로 호출한다.

`content` 필드와 `tool_calls` 필드 모두 확인 가능하다.

In [11]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: Hello! How can I assist you today?
ToolCalls: []


툴 호출이 필요한 입력으로 호출한다.

In [12]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Seoul?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ewXMYMg4borP5KvUzMooSuQE'}]


## 에이전트 생성
툴과 LLM을 정의했으므로 에이전트를 생성할 수 있다.

여기서는 [LangGraph](https://langchain-ai.github.io/langgraph/)를 사용하여 에이전트를 구성한다.
`LangGraph`를 사용하면 간편하게 로직을 수정할 수 있다.

In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## 에이전트 실행
에이전트를 실행합니다. 에이전트는 상호작용의 최종 상태를 반환한다.

In [14]:
# 툴 호출 필요없을 때 응답
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='ffbd474b-433c-44f2-a4d1-ac5978f473e1'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f4e629d0a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-909a891c-38e8-46f9-b8c7-22d047cd9f88-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [15]:
# 툴 호출 필요할 때 응답
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Seoul?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='fcc72f43-5848-4039-afbb-d60a68388bfa'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EOgOK9XsxWyjcZnMPojAP3cU', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f4e629d0a5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d28080a8-6cd2-44be-b948-c97c53805dc5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_EOgOK9XsxWyjcZnMPojAP3cU'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of Am

## 메시지 스트리밍
`.invoke`를 사용하여 최종 응답만을 가져오면 에이전트가 여러 단계를 실행할 때 오랜 시간을 기다리게 된다.

중간 진행 상황을 표시하여 메시지를 스트리밍 할 수 있다.

In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Seoul?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZsmaWJjRiWFRAl3150TrfnPb', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f4e629d0a5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9a193f9d-d69e-44d9-bc8d-c7b10532b982-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ZsmaWJjRiWFRAl3150TrfnPb'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42

## 토큰 스트리밍
메시지를 스트리밍하는 것 외에도 토큰을 스트리밍하는 것도 유용하다.

`.astream_events` 메서드를 사용하여 이를 수행할 수 있다.

In [17]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in Seoul?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # 에이전트를 `.with_config({"run_name": "Agent"})`로 생성할 때 지정됨
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # 에이전트를 `.with_config({"run_name": "Agent"})`로 생성할 때 지정됨
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # OpenAI의 문맥에서 비어 있는 내용은
            # 모델이 호출할 도구를 요청하고 있음을 의미합니다.
            # 따라서 빈 내용이 아닌 내용만 인쇄합니다.
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

c:\Users\강영인\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718700624, 'localtime': '2024-06-18 1:50'}, 'current': {'last_updated_epoch': 1718700300, 'last_updated': '2024-06-18 01:45', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 70, 'cloud': 25, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 11.9, 'windchill_f': 53.4, 'heatindex_c': 12.2, 'heatindex_f': 53.9, 'dewpoint_c': 

## 메모리 추가
지금까지 구현 한 내용으로는 이전 대화를 기억하지 않기 때문에 자연스러운 대화가 이어질 수 없다.

메모리를 제공하기 위해 체크포인터를 전달한다.

`thread_id`를 전달하여 이어지는 대화인지, 새로운 대화인지 구분한다.

In [18]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [19]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello, Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 83, 'total_tokens': 95}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f4e629d0a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-b9273cb6-2ed6-46a5-982e-fbbb94a0d31a-0', usage_metadata={'input_tokens': 83, 'output_tokens': 12, 'total_tokens': 95})]}}
----


In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 107, 'total_tokens': 120}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_9cb5d38cf7', 'finish_reason': 'stop', 'logprobs': None}, id='run-90d90ef2-55e8-4173-8118-d8d2f09864ae-0', usage_metadata={'input_tokens': 107, 'output_tokens': 13, 'total_tokens': 120})]}}
----


새로운 대화를 시작할 때는 `thread_id`를 변경한다.

In [22]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="You haven't provided your name. Could you please tell me your name?", response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 84, 'total_tokens': 99}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_9cb5d38cf7', 'finish_reason': 'stop', 'logprobs': None}, id='run-13a12f24-69c2-4a3e-adc7-82e55ea5a212-0', usage_metadata={'input_tokens': 84, 'output_tokens': 15, 'total_tokens': 99})]}}
----
